GOAL:

- Create points using JSON URL from Open Data site
- Publish the points to ArcGIS Online or ArcGIS Portal

Inspired by:

- [Bryan McIntosh - Export ArcGIS Server Map Service Layer to Shapefile](http://www.spatialtimes.com/2016/03/extract-map-service-layer-shapefile-using-python/)
- [Corey Schafer python JSON YouTube video](https://youtu.be/9N6a-VLBa2I)
- [Corey Schafer GitHub](https://github.com/CoreyMSchafer/code_snippets/blob/master/Python-JSON/api.py)

In [1]:
import urllib3
import json
import arcpy
import pandas as pd
import numpy as np

[urllib3 SSL Warnings](https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings)

In [ ]:
urllib3.disable_warnings()

In [3]:
jsonURL = "https://data.cityofgainesville.org/resource/c2ek-5hm9.json"
#jsonURL = 'https://data.cityofgainesville.org/api/views/f4t5-t3b5/rows.json'
#geojsonURL = 'https://data.cityofgainesville.org/resource/pqg3-m2ek.geojson'
gdb = r"C:\Users\friendde\Documents\ArcGIS\Projects\MIMS\MIMS.gdb"
#fc = r"C:\Users\friendde\Documents\ArcGIS\Projects\MIMS\MIMS.gdb\ActiveBusiness\ActiveBusinessCity"
tbl = r"C:\Users\friendde\Documents\ArcGIS\Projects\MIMS\MIMS.gdb\BusinessCity"
#flds = ['BusinessName','BusinessPhone','BusinessType','BusinessAddress','BusinessOwner','POINT_X','POINT_Y','SHAPE@XY']
flds = ['BusinessName','BusinessPhone','BusinessType','BusinessAddress','BusinessOwner','POINT_X','POINT_Y']
location = [0,0]
tupleXY = (0,0)

In [4]:
arcpy.env.workspace = gdb
arcpy.env.overwriteOutput = True
#arcpy.env.outputCoordinateSystem = arcpy.SpatialReference(4326)

In [5]:
def getCoords(_dict):
    if _dict is None:
        return [0,0]
    else:
        #print(_dict.get('coordinates'))
        return _dict.get('coordinates')

In [6]:
http = urllib3.PoolManager()
response = http.request('GET', jsonURL)
#response.status
#response.data

In [7]:
data = json.loads(response.data)

In [ ]:
#print(json.dumps(data, indent=2))

In [ ]:
arcpy.TruncateTable_management(tbl)

In [ ]:
# use try/except to catch keyerror
for item in data:
    try:
        cursor = arcpy.da.InsertCursor(tbl,flds)
        bizowner = item['owner']
        bizname = item['name']
        bizphone = item['business_phone']
        biztype = item['business_type']
        bizaddress = item['location_address']
        location = getCoords(item['location'])
        locationX = location[0]
        locationY = location[1]
        tupleXY = (locationX,locationY)
        #print(bizname,'\n','\t',bizphone,'\n','\t',biztype,'\n','\t',bizaddress,'\n','\t',locationX,locationY)
        #business = [bizname,bizphone,biztype,bizaddress,bizowner,locationX,locationY,tupleXY]
        business = [bizname,bizphone,biztype,bizaddress,bizowner,locationX,locationY]
        print(business)
        cursor.insertRow(business)
        del cursor
    except KeyError:
        location = [0,0]
        tupleXY = (0,0)
        continue


### Use numpy and pandas to export to CSV
 
Use arcpy [```TableToNumPyArray()```](http://pro.arcgis.com/en/pro-app/arcpy/data-access/tabletonumpyarray.htm)
See also [Working with numpy in ArcGIS](http://pro.arcgis.com/en/pro-app/arcpy/get-started/working-with-numpy-in-arcgis.htm)


#nparr = arcpy.da.TableToNumPyArray(tbl,flds,skip_nulls=True)
#pdarr = pd.DataFrame(nparr)
#pdarr.to_csv(custAcctFile,header=False, index=False)

Ready new numpy array for consumption analysis

In [8]:
nparr = arcpy.da.TableToNumPyArray(tbl,flds,skip_nulls=True)
df = pd.DataFrame(nparr)
df.head(10)

,BusinessName,BusinessPhone,BusinessType,BusinessAddress,BusinessOwner,POINT_X,POINT_Y
0,"EARLY JONES, MAXINE",352-338-0450,MASSAGE THERAPIST,3366 NW 13TH ST,MAXINE E JONES,-82.339120,29.683756
1,"LLOYD CLARKE'S RACQUET SHOPPE, INC.",352-372-7836,RETAIL MERCHANT,1504 NW 13TH ST,"LLOYD CLARKE'S RACQUET SHOPPE, INC.",-82.339161,29.666101
2,"CPA MUTUAL MANAGEMENT, INC.",352-240-7800,BUSINESS OFFICE NOT OTHERWISE TAXED,04923 NW 43RD ST,"CPA MUTUAL INSURANCE COMPANY OF AMERICA, RRG",-82.388901,29.700474
3,ADVANCED TAX SERVICES,407-694-7978,BOOKKEEPING OR INCOME TAX SERVICE,"1136 NE 8TH AVE, SUITE B",ADVANCED TAX SERVICES INC,-82.310528,29.659321
4,"DREYFUS, JOYCE",352-373-4800,PROFESSIONAL-ACUPUNCTURE,"2251 NW 41ST ST, SUITE 20",JOYCE DREYFUS,-82.386251,29.673933
5,KGM NETWORKING LLC,352-278-0251,CONTRACTOR-INSTALLATION,3637 NE 2ND ST,KGM NETWORKING LLC,-82.322395,29.686546
6,PEAK STRENGTH & FITNESS LLC,352-317-2484,PROFESSIONAL NOT OTHERWISE LICENSED,1947 N MAIN ST,"PEAK STRENGTH & FITNESS, LLC",-82.323469,29.670423
7,XIE OCEAN BUFFET INC,352-331-1700,RESTAURANT,"6795 W NEWBERRY RD, # 7B",XIE OCEAN BUFFET INC,-82.414411,29.659587
8,BORCOR FOODS INC,352-225-3539,RESTAURANT,"201 SE 1ST ST, UNIT 106",BORCOR FOODS INC,-82.323959,29.650268
9,"ALL TYPE VACUUM CTR, INC",352-372-1000,REPAIRS NOT REQUIRING STATE LICENSE,2225 NW 6TH ST,"ALL TYPE VACUUM CTR, INC",-82.330791,29.673669


In [ ]:
df.groupby('BusinessType')['BusinessType'].value_counts()

#pivot table
pivotTbl = df.pivot_table(values=["BusinessType"], index=["BusinessType"], aggfunc='count')
pivotTbl

[```gis.features.SpatialDataFrame()```](https://esri.github.io/arcgis-python-api/apidoc/html/arcgis.features.toc.html?highlight=spatialdataframe#arcgis.features.SpatialDataFrame.from_xy)

In [9]:
from arcgis.features import SpatialDataFrame
from arcgis.gis import GIS
from getpass import getpass
from IPython.display import display

In [23]:
sdf = SpatialDataFrame.from_xy(df,"POINT_X","POINT_Y")
display(sdf)

,BusinessName,BusinessPhone,BusinessType,BusinessAddress,BusinessOwner,POINT_X,POINT_Y,SHAPE
0,"EARLY JONES, MAXINE",352-338-0450,MASSAGE THERAPIST,3366 NW 13TH ST,MAXINE E JONES,-82.339120,29.683756,"{'x': -82.33912, 'y': 29.683756, 'spatialRefer..."
1,"LLOYD CLARKE'S RACQUET SHOPPE, INC.",352-372-7836,RETAIL MERCHANT,1504 NW 13TH ST,"LLOYD CLARKE'S RACQUET SHOPPE, INC.",-82.339161,29.666101,"{'x': -82.339161, 'y': 29.666101, 'spatialRefe..."
2,"CPA MUTUAL MANAGEMENT, INC.",352-240-7800,BUSINESS OFFICE NOT OTHERWISE TAXED,04923 NW 43RD ST,"CPA MUTUAL INSURANCE COMPANY OF AMERICA, RRG",-82.388901,29.700474,"{'x': -82.388901, 'y': 29.700474, 'spatialRefe..."
3,ADVANCED TAX SERVICES,407-694-7978,BOOKKEEPING OR INCOME TAX SERVICE,"1136 NE 8TH AVE, SUITE B",ADVANCED TAX SERVICES INC,-82.310528,29.659321,"{'x': -82.310528, 'y': 29.659321, 'spatialRefe..."
4,"DREYFUS, JOYCE",352-373-4800,PROFESSIONAL-ACUPUNCTURE,"2251 NW 41ST ST, SUITE 20",JOYCE DREYFUS,-82.386251,29.673933,"{'x': -82.386251, 'y': 29.673933, 'spatialRefe..."
5,KGM NETWORKING LLC,352-278-0251,CONTRACTOR-INSTALLATION,3637 NE 2ND ST,KGM NETWORKING LLC,-82.322395,29.686546,"{'x': -82.322395, 'y': 29.686546, 'spatialRefe..."
6,PEAK STRENGTH & FITNESS LLC,352-317-2484,PROFESSIONAL NOT OTHERWISE LICENSED,1947 N MAIN ST,"PEAK STRENGTH & FITNESS, LLC",-82.323469,29.670423,"{'x': -82.323469, 'y': 29.670423, 'spatialRefe..."
7,XIE OCEAN BUFFET INC,352-331-1700,RESTAURANT,"6795 W NEWBERRY RD, # 7B",XIE OCEAN BUFFET INC,-82.414411,29.659587,"{'x': -82.414411, 'y': 29.659587, 'spatialRefe..."
8,BORCOR FOODS INC,352-225-3539,RESTAURANT,"201 SE 1ST ST, UNIT 106",BORCOR FOODS INC,-82.323959,29.650268,"{'x': -82.323959, 'y': 29.650268, 'spatialRefe..."
9,"ALL TYPE VACUUM CTR, INC",352-372-1000,REPAIRS NOT REQUIRING STATE LICENSE,2225 NW 6TH ST,"ALL TYPE VACUUM CTR, INC",-82.330791,29.673669,"{'x': -82.330791, 'y': 29.673669, 'spatialRefe..."


In [24]:
#gis = GIS("https://wms.gru.com/portal", username=input("Enter User Name "), password=(getpass()))
gis = GIS(arcpy.GetActivePortalURL(), username=input("Enter User Name "), password=(getpass()))
#gis = GIS()
#portalDesc = arcpy.GetPortalDescription()
# search and list all items owned by connected user
#query=f'owner:{portalDesc["user"]["username"]} AND title:CW BaseMap'
#itemType="Feature Layer"
#sortField="title"
#sortOrder="asc"
# default max__items is 10
#maxItems=100
#m = gis.content.search(query,itemType,sortField,sortOrder,maxItems)


Enter User Name arcgiswebservices
········


In [25]:
bizLayer = gis.content.import_data(sdf)

In [19]:
m = gis.map('Gainesville,FL')

In [20]:
m

MapView(basemaps=['dark-gray', 'dark-gray-vector', 'gray', 'gray-vector', 'hybrid', 'national-geographic', 'oc…

In [27]:
m.add_layer(bizLayer)
m

MapView(basemaps=['dark-gray', 'dark-gray-vector', 'gray', 'gray-vector', 'hybrid', 'national-geographic', 'oc…